# Equipment Assignment (IP)

This problem is taken from Prof Jack Ponton's [Modelling, Simulation and Optimisation Online Course](https://www.homepages.ed.ac.uk/jwp/newMSOcopy/). Specifically, the LP relaxation is discussed in [Section 5.4.2: Linear Programming Problems](https://www.homepages.ed.ac.uk/jwp/newMSOcopy/section5/lp.html#batch) and the IP is discussed in [Section 5.4.3: Mixed Integer Linear Programming (MILP) Problems](https://www.homepages.ed.ac.uk/jwp/newMSOcopy/section5/milp.html).

A company wants to optimize profit through the sales of products (A, B, C)

**Processing Time (in hours)**

| Product ↓ / Process →   | Reactor | Crystalliser | Centrifuge |
|-------------------------|---------|--------------|------------|
| A                       | 0.8     | 0.4          | 0.2        |
| B                       | 0.2     | 0.3          | -          |
| C                       | 0.3     | -            | 0.1        |

**Available Process Time (in minutes)**

| Process ↓   | Time  |
|-------------|-------|
| Reactor     | 40    |
| Crytallizer | 35    |
| Centrifuge  | 35    |

**Profit on Sales (in $)**

| Product ↓   | Profit  |
|-------------|---------|
| A           | 20      |
| B           | 6       |
| C           | 8       |

**Sales limit (in units)**

| Product ↓   | Limit   |
|-------------|---------|
| C           | 20      |



## General Definition

A function is made to handle the general definition of the problem, since the indices and most parameters remain the same, irrespective of how we treat the variables 

In [8]:
from gana import Prg, V, P, I, sup


def general_def(name: str, itg: bool = True, rhs: list[int] = [20, 10, 5]) -> Prg:
    p = Prg(name)
    p.products = I('a', 'b', 'c')
    p.processes = I('reactor', 'crystalliser', 'centrifuge')
    p.profit = P(p.products, _=[20, 6, 8])
    p.time = P(p.products, p.processes, _=[0.8, 0.4, 0.2, 0.2, 0.3, 0, 0.3, 0, 0.1])
    p.tmax = P(p.processes, _=rhs)
    p.nbatch = V(p.products, itg=itg)
    p.cons_time = (
        sum(p.time(product, p.processes) * p.nbatch(product) for product in p.products)
        <= p.tmax
    )
    p.sales_lm = p.nbatch(p.c) <= 20
    p.o = sup(sum(p.profit(product) * p.nbatch(product) for product in p.products))
    return p

## LP Relaxation 

Strictly speaking the number of batches processed should be a integer values. Nevertheless, let us solve the relaxed problem first. 




In [9]:
p1 = general_def(name='lp_rlx', itg=False)
p1.show()

# Mathematical Program for lp_rlx

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<br><br>

## Functions

<IPython.core.display.Math object>

Optimizing the program

In [10]:
p1.opt()

📝  Generated lp_rlx.mps                                                     ⏱ 0.0007 s


Read MPS format model from file lp_rlx.mps
Reading time = 0.00 seconds
LP_RLX: 4 rows, 3 columns, 8 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0024 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 4 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x6d41737c
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.0000000e+02   5.331617e+01   0.000000e+00      0s
       2   -5.2500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective -5.250000000e+02


📝  Generated Solution object for lp_rlx. See .solution                      ⏱ 0.0000 s
✅  lp_rlx optimized using gurobi. Display using .output()                   ⏱ 0.0102 s


The number of batches (```nbatch```) have non-integer solutions.

In [11]:
p1.nbatch.output(asdict=True)

{(a,): 13.749999999999998, (b,): 15.0, (c,): 20.0}

With an objective value of:

In [12]:
p1.o.output(asfloat=True)

-525.0

## IP Problem

In [13]:
p2 = general_def(name='IP')
p2.opt()

📝  Generated IP.mps                                                         ⏱ 0.0007 s


Read MPS format model from file IP.mps
Reading time = 0.00 seconds
IP: 4 rows, 3 columns, 8 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0028 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 4 rows, 3 columns and 8 nonzeros
Model fingerprint: 0xaaa99143
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Found heuristic solution: objective -500.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 7 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective -5.250000e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -525.000

📝  Generated Solution object for IP. See .solution                          ⏱ 0.0000 s
✅  IP optimized using gurobi. Display using .output()                       ⏱ 0.0124 s


The solution obtained from the LP relaxation is pretty close to the actual integer solution, with a lower objective value

In [14]:
p2.output()

# Solution for IP

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

## Variables

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>